In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

from Parameters import ModelParams
from Mass_Loss_Class import MassLoss
from Fractionation_Class import Fractionation
from Model_Data_Loader import ModelDataLoader
from Results_Handler import ResultsHandler
from Misc_Class import Misc

In [ ]:
%matplotlib widget

In [ ]:
def main():

    # ---------- Initialize parameters and classes ----------
    params = ModelParams()
    base_path = '/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/CW/critical_WMF/gridmin295/'
    data_loader = ModelDataLoader(base_path)
    mass_loss = MassLoss(params)
    fractionation = Fractionation(params)
    misc = Misc(params)

    # ---------- Load data ----------
    m_planet = [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    water_percent = [x / 10 for x in range(1, 81)]
    radius, mass, Teq = data_loader.load_model_data(m_planet, water_percent)
    
    # ---------- OR select a planet and also change the EUV flux accordingly ----------
    # ---------- OR manualy set these parameters ----------

    # with open("planet_params.json", "r") as file:
    #     planet_params = json.load(file)
    # selected_planet = "Kepler-36 c"

    # if selected_planet in planet_params:
    #     new_feuv = planet_params[selected_planet]["FEUV"]
    #     params.update_param("FEUV", new_feuv)
    #     print(f"Updated FEUV for {selected_planet}: {params.get_param('FEUV'):.2e} erg cm⁻² s⁻¹")
    # else:
    #     print(f"Make sure the planet you selected exists in the list! Check the spelling.")

    # mearth = params.mearth
    # rearth = params.rearth
    # radius = [planet_params[selected_planet]["radius"] * rearth] # cm
    # mass = [planet_params[selected_planet]["mass"] * mearth] # g
    # Teq = [planet_params[selected_planet]["teq"]] # K

    # ---------- Mass loss model ----------
    mass_loss_results = mass_loss.compute_mass_loss_parameters(mass, radius, Teq)

    # ---------- Fractionation model ----------
    # fractionation_results = fractionation.execute_fractionation(mass_loss_results, misc)

    # ---------- Fractionation model with feedback loop ----------
    fractionation_results = fractionation.execute_self_consistent_fractionation(mass_loss_results, mass_loss, misc, params)

    ResultsHandler.plot_cs_REUV(fractionation_results)
    # ResultsHandler.plot_combined_T_P(fractionation_results)
    ResultsHandler.plot_phiH_phiO_vs_REUV(fractionation_results)
    # ResultsHandler.plot_radii_comparison(fractionation_results)
    # ResultsHandler.water_loss_over_time(fractionation_results)

if __name__ == "__main__":
   main()